In [237]:
import random
import ipywidgets as widgets
from IPython import display
import time
import pdb

In [238]:
# values to use lates to build the shoe

suits = ('Hearts', 'Diamonds', 'Spades', 'Clubs')
ranks = ('Two','Three','Four','Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten', 'Jack', 'Queen', 'King','Ace')
values = {'Two':2, 'Three':3, 'Four':4, 'Five':5, 'Six':6, 'Seven':7, 'Eight':8,'Nine':9, 'Ten':10, 'Jack':10, 
         'Queen':10, 'King':10, 'Ace':11}

In [239]:
class Card:
    
    def __init__(self, suit, rank):
        # creates the card, with its value in self.value
        self.suit = suit
        self.rank = rank
        self.value = values[rank]
    
    def __str__(self):
        # creates a print to show the player via text
        return self.rank + ' of ' + self.suit

In [240]:
class Shoe:
    
    def __init__(self):
        # creates class shoe with 4 decks 
        self.shoe = []
        i = 1
        while i <= 4:
            for suit in suits:
                for rank in ranks:
                    card = Card(suit, rank)
                    self.shoe.append(card)
            i += 1
    
    def __str__(self):
        # just to check if shoe created correctly
        shoe_comp = ''
        for card in self.shoe:
            shoe_comp += '\n' + card.__str__()
        return 'The shoe has:' + shoe_comp
    
    def shuffle(self):
        # randomly shuffles the shoe
        random.shuffle(self.shoe)
        
    def deal(self):
        # takes one card from the shoe
        return self.shoe.pop()
    
    def shoelist(self):
        # to check the list and how many cards are in the shoe
        return self.shoe

In [241]:
class Hand:
    def __init__(self):
        # creates hand, with value and keep track of aces for 1 or 11 value and for 'soft' or 'hand' value
        self.cards = []  
        self.value = 0   
        self.aces = 0    
    
    def add_card(self,card):
        # when card taken, add a card and the value. If card is ace, adds one ace to self.aces
        self.cards.append(card)
        self.value += card.value
        if card.rank == 'Ace':
            self.aces += 1
    
    def adjust_for_ace(self):
        # adjusts the value of one ace to 1 if total is more than 21 and takes one ace out from tracking
        while self.value > 21 and self.aces:
            self.value -= 10
            self.aces -= 1

In [242]:
class Chips:
    
    def __init__(self):
        # creates the chips. By default, player starts with 100 but that could be changed in future recoding
        self.total = 100
        self.bet = 0
        
    def win_bet(self):
        # in case of wining, chips are increased by 1/1 gain/bet 
        self.total += self.bet
    
    def lose_bet(self):
        # in case of losing, chips are decreased by 1/1 loss/bet
        self.total -= self.bet
        
    def blackjack_bet(self):
        # in case of winning with blackjack, chips are increased by 3/2 gain/bet
        self.total = int(self.total*1.5)

In [243]:
def initialize_game():
    global shoe, player, dealer, slidera, buttonc, ft
    # ft means first time, in that case, creates a new shoe
    if ft:
        shoe = Shoe()
        shoe.shuffle()
    # also, if 25% or less remaining cards in the show, creates a new shoe
    elif len(shoe.shoelist()) <= 52:
        shoe = Shoe()
        shoe.shuffle()
        print('Shoe has been shuffled')
    
    #creates new hands for player, dealer, takes two card for each and adjust for aces
    player = Hand()
    dealer = Hand()
    player.add_card(shoe.deal())
    player.add_card(shoe.deal())
    player.adjust_for_ace()
    dealer.add_card(shoe.deal())
    dealer.add_card(shoe.deal())
    dealer.adjust_for_ace()
    
    display.clear_output()
    
    # first time welcomes the player, then sets first time to false
    if ft:
        print('Welcome to BlackJack!')
        ft = False
    
    # creates a slider from 1 to maximum of chips to ask for a bet,then press the button to bet
    print('How much do you want to bet?')
    print(f'You have {chips.total} chips')
    slidera = widgets.IntSlider(value=(chips.total)/3,min=1,max=chips.total)
    buttonc = widgets.Button(description='Make Bet',button_style='info') 
    display.display(slidera)
    display.display(buttonc)
    buttonc.on_click(on_button_clicked_c)
    

In [244]:
def take_bet():
    global slidera, buttonc, bet, player, dealer
    #closes the slider, prints the value being bet and starts the player rounds
    chips.bet = slidera.value
    slidera.close()
    buttonc.close()
    display.clear_output()
    print(f'You are betting {slidera.value} chips')
    player_rounds()
    


In [345]:
def show_some(player,dealer):
    #creates a list to print the cards in hand and adds 'soft' or 'hard' before the value if needed
    global insurance_card
    
    listplayer = []
    for i in player.cards:
        listplayer.append(i)
    print('\nYou have the following cards:\n')
    for i in listplayer:
        print(i)
    # find if soft or hard
    if 12 <= player.value <= 20 and player.aces == 1:
        print(f"Value = 'Soft' {player.value}")
    elif 12 <= player.value <= 20 and player.aces == 0:
        print(f"Value = 'Hard' {player.value}")
    else:
        print(f"Value = {player.value}")
    
    # same for the dealer but no need of 'soft' or 'hard'. Instead, keeps one of the cards hidden 
    listdealer = []
    for i in dealer.cards:
        listdealer.append(i)
    print('\n\nDealer has the following cards:\n')
    print('Hole card')
    for i in listdealer[1::]:
        insurance_card = i
        print(i)
        holevalue = i.value
    print(f'Value = {holevalue} + X')
    print('\n\n')

In [246]:
def show_all(player,dealer):
    
    # to use when round ends, just a list of cards in both hands
    
    listplayer = []
    for i in player.cards:
        listplayer.append(i)
    
    listdealer = []
    for i in dealer.cards:
        listdealer.append(i)
        
    print(f'\nYour value is {player.value} and has the following cards:\n')
    for i in listplayer:
        print(i)
    print(f'\n\nDealer value is {dealer.value} and has the following cards:\n')
    for i in listdealer:
        print(i)
    print('\n\n')

In [247]:
def player_rounds():
    global player, shoe, dealer
    
    # checks every time for different scenarios: blackjack, bust, 21. If not one of them, asks player next move
    if player.value == 21 and len(player.cards) == 2 and dealer.value != 21:
        display.clear_output()
        blackjack_win(player, chips)
    elif player.value < 21:
        hit_or_stand()
    elif player.value > 21:
        display.clear_output()
        player_busts(player, chips)
    elif player.value == 21 and len(player.cards) != 2:
        display.clear_output()
        dealer_rounds()
    

In [248]:
def dealer_rounds():
    global player, shoe, dealer, chips, bet
    
    # dealer takes cards until value reaches 17 or more, then stops
    while dealer.value < 17 :
        hit(shoe, dealer)
        
    # checks different ending scenarios: win, bust, lost, push
    if player.value > dealer.value:
        display.clear_output()
        player_wins(player,dealer,chips)
    elif dealer.value > player.value and dealer.value < 21:
        display.clear_output()
        dealer_wins(player, dealer, chips)
    elif dealer.value > 21:
        display.clear_output()
        dealer_busts(player, dealer, chips)
    elif player.value == dealer.value:
        display.clear_output()
        push(player, dealer)
    elif dealer.value == 21:
        display.clear_output()
        dealer_wins(player, dealer, chips)

In [352]:
def hit_or_stand():
    global buttond,buttone, player, dealer, buttonh, buttonj, buttonm, buttonp, insurance_card
    
    # with buttons, asks the player next move. Hit, stand or double down
    show_some(player, dealer)
    buttond = widgets.Button(description='Hit',button_style='info')
    buttone = widgets.Button(description='Stand',button_style='info')
    buttonh = widgets.Button(description='Double Down',button_style='info')
    display.display(buttond)
    display.display(buttone)
    display.display(buttonh)
    buttond.on_click(on_button_clicked_d)
    buttone.on_click(on_button_clicked_e)
    buttonh.on_click(on_button_clicked_h)
    
    # if first round is a pair of anything but aces, it shows the button to split
    if len(player.cards) == 2 and player.aces == 0 and (player.cards)[0].value == (player.cards[1]).value:
        buttonj = widgets.Button(description='Split',button_style='info')
        display.display(buttonj)
        buttonj.on_click(on_button_clicked_j)
        
    # if first round, option to surrender
    if len(player.cards) == 2:
        buttonm = widgets.Button(description='Surrender',button_style='info')
        display.display(buttonm)
        buttonm.on_click(on_button_clicked_m)
        
    # if dealer is showing and ace, gives the option to bet that dealer has a blackjack, pays 2:1
    
        

In [250]:
def hit(shoe,hand):
    
    # takes a card from the shoe and adds it to the selected hand
    hitcard = shoe.deal()
    hand.add_card(hitcard)
    hand.adjust_for_ace()

In [251]:
def double_down_hit():
    global buttoni, chips
    
    # doubles the bet and shows button to hit
    chips.bet = (chips.bet)*2
    print(f'Bet has been doubled. Now betting {chips.bet} chips')
    show_some(player, dealer)
    buttoni = widgets.Button(description='Hit',button_style='info')
    display.display(buttoni)
    buttoni.on_click(on_button_clicked_i)
    
    
    

In [252]:
def double_down_stand(shoe, player):
    global dealer, chips, buttone
    
    # cheks for bust and if not, shows stand button
    if player.value > 21:
        display.clear_output()
        player_busts(player, chips)
    
    elif player.value == 21:
        display.clear_output()
        dealer_rounds()
    
    elif player.value < 21:
        display.clear_output()
        show_some(player,dealer)
        buttone = widgets.Button(description='Stand',button_style='info')
        display.display(buttone)
        buttone.on_click(on_button_clicked_e)
    

In [253]:
def player_wins(player, dealer, chips):
    
    # prints that playes has won and adds bet to the player's chips. Then asks to play again or not
    if player.value == 21:
        print(f"You win! Your cards value is exactly 21 and Dealer's cards value is {dealer.value}")
        chips.win_bet()
        print(f'You have now {chips.total} chips')
        ask_for_playagain()
    
    elif player.value > dealer.value and player.value < 21:
        print(f"You win! Your cards value is {player.value} and Dealer's cards value is {dealer.value} ")
        chips.win_bet()
        print(f'You have now {chips.total} chips')
        ask_for_playagain()

In [254]:
def blackjack_win(player, chips):
    
    # prints blackjack win and adds the special bet price to player's chips. Then asks to play again or not
    print('Blackjack! You win 3/2')
    chips.blackjack_bet()
    ask_for_playagain()

In [255]:
def player_busts(player,chips):
    
    # prints that player has lost and remove the bet from player's chips. Then asks to play again or not
    print(f"You lose. Your cards value is {player.value}. Dealer wins")
    chips.lose_bet()
    print(f'You have  now {chips.total} chips')
    ask_for_playagain()

In [256]:
def dealer_wins(player, dealer, chips):
    
    # prints that dealer won and adjust the player's chips. Then asks to play again or not
    if dealer.value == 21:
        print("Dealer wins. Dealer's cards value is exactly 21.")
        chips.lose_bet()
        print(f'You have now {chips.total} chips')
        ask_for_playagain()
    
    elif dealer.value > player.value and dealer.value < 21:
        print(f"You lose. Your cards value is {player.value} and Dealer's cards value is {dealer.value}")
        chips.lose_bet()
        print(f'You have now {chips.total} chips')
        ask_for_playagain()

In [257]:
def dealer_busts(player, dealer, chips):
    
    # prints that dealer busted and adjusts the chips. Then asks to play again or not
    print(f"You win! Your cards value is {player.value} and Dealer's cards value is {dealer.value}")
    chips.win_bet()
    print(f'You have now {chips.total} chips')
    ask_for_playagain()

In [258]:
def push(player, dealer):
    
    # prints player and dealer tied and then asks to play again or not
    print(f'Push! You and Dealer tie.{player.value}/{dealer.value} Bet is returned')
    print(f'You have {chips.total} chips')
    ask_for_playagain()

In [259]:
def ask_for_playagain():
    global buttonf, buttong
    
    # if no more chips, game is over, else, asks the player to play again or not
    if chips.total == 0:
        print('GAME OVER. You ran out of chips. Thanks for playing')
    else:
        print('\nPlay again?')
        buttonf = widgets.Button( description='Yes', button_style='info')
        buttong = widgets.Button( description='No', button_style='info')
        display.display(buttonf)
        display.display(buttong)
        buttonf.on_click(on_button_clicked_f)
        buttong.on_click(on_button_clicked_g)

In [260]:
def player_rounds_main():
    global player, shoe, dealer
    
    if player.value >= 21:
        display.clear_output()
        player_rounds_secondary()
    elif player.value < 21:
        hit_or_stand_player()
   

In [261]:
def player_rounds_secondary():
    global player, secondaryplayer, shoe, dealer
    
    if secondaryplayer.value == 21:
        display.clear_output()
        dealer_rounds_split()
    elif secondaryplayer.value > 21:
        display.clear_output()
        dealer_rounds_split()
    elif secondaryplayer.value < 21:
        display.clear_output()
        hit_or_stand_secondaryplayer()

In [262]:
def dealer_rounds_split():
    global dealer, shoe
    
    # dealer takes cards until value reaches 17 or more, then stops
    while dealer.value < 17 :
        hit(shoe, dealer)
    show_all_split()
    checks_endround_split()

In [302]:
def hit_or_stand_player():
    global buttonk, buttonl, buttonm
    
    # with buttons, asks the main player next move. Hit, stand or double down
    show_some_split()
    print('PLAYING MAIN HAND')
    buttonk = widgets.Button(description='Hit',button_style='info')
    buttonl = widgets.Button(description='Stand',button_style='info')
    display.display(buttonk)
    display.display(buttonl)
    buttonk.on_click(on_button_clicked_k)
    buttonl.on_click(on_button_clicked_l)

In [301]:
def hit_or_stand_secondaryplayer():
    global buttonn, buttono, buttonp
    
    #with buttons, asks the split player next move. Hit, stand or double down
    show_some_split()
    print('PLAYING SPLIT HAND')
    buttonn = widgets.Button(description='Hit',button_style='info')
    buttono = widgets.Button(description='Stand',button_style='info')
    display.display(buttonn)
    display.display(buttono)
    buttonn.on_click(on_button_clicked_n)
    buttono.on_click(on_button_clicked_o)

In [265]:
def show_some_split():
    global player, secondaryplayer, dealer
    
    #creates a list to print the cards in hand and adds 'soft' or 'hard' before the value if needed
    listplayer = []
    for i in player.cards:
        listplayer.append(i)
    print('\nYour main hand has the following cards:\n')
    for i in listplayer:
        print(i)
    # find if soft or hard
    if 12 <= player.value <= 20 and player.aces == 1:
        print(f"Value = 'Soft' {player.value}")
    elif 12 <= player.value <= 20 and player.aces == 0:
        print(f"Value = 'Hard' {player.value}")
    else:
        print(f"Value = {player.value}")
    
    
    #repeat process to show secondary hand( split hand)
    listsecondaryplayer = []
    for i in secondaryplayer.cards:
        listsecondaryplayer.append(i)
    print('\nYour split hand has the following cards:\n')
    for i in listsecondaryplayer:
        print(i)
    
    if 12 <= secondaryplayer.value <= 20 and secondaryplayer.aces == 1:
        print(f"Value = 'Soft' {secondaryplayer.value}")
    elif 12 <= secondaryplayer.value <= 20 and secondaryplayer.aces == 0:
        print (f"Value = 'Hard' {secondaryplayer.value}")
    else:
        print(f"Value = {secondaryplayer.value}")
    
    
    # same for the dealer but no need of 'soft' or 'hard'. Instead, keeps one of the cards hidden 
    listdealer = []
    for i in dealer.cards:
        listdealer.append(i)
    print('\n\nDealer has the following cards:\n')
    print('Hole card')
    for i in listdealer[1::]:
        print(i)
        holevalue = i.value
    print(f'Value = {holevalue} + X')
    print('\n\n')

In [297]:
def show_all_split():
    global player, secondaryplayer, dealer
    
    #creates a list to print the cards in hand and adds 'soft' or 'hard' before the value if needed
    listplayer = []
    for i in player.cards:
        listplayer.append(i)
    print('\nYour main hand has the following cards:\n')
    for i in listplayer:
        print(i)
    # find if soft or hard
    if 12 <= player.value <= 20 and player.aces == 1:
        print(f"Value = 'Soft' {player.value}")
    elif 12 <= player.value <= 20 and player.aces == 0:
        print(f"Value = 'Hard' {player.value}")
    else:
        print(f"Value = {player.value}")
    
    
    #repeat process to show secondary hand( split hand)
    listsecondaryplayer = []
    for i in secondaryplayer.cards:
        listsecondaryplayer.append(i)
    print('\nYour split hand has the following cards:\n')
    for i in listsecondaryplayer:
        print(i)
    
    if 12 <= secondaryplayer.value <= 20 and secondaryplayer.aces == 1:
        print(f"Value = 'Soft' {secondaryplayer.value}")
    elif 12 <= secondaryplayer.value <= 20 and secondaryplayer.aces == 0:
        print (f"Value = 'Hard' {secondaryplayer.value}")
    else:
        print(f"Value = {secondaryplayer.value}")
    
    
    # same for the dealer but no need of 'soft' or 'hard'. 
    listdealer = []
    for i in dealer.cards:
        listdealer.append(i)
    print('\n\nDealer has the following cards:\n')
    for i in listdealer:
        print(i)
    print(f'Value = {dealer.value}')
    print('\n\n')

In [333]:
def checks_endround_split():
    global chips, player, secondaryplayer, dealer
    
    print('')
    
    if dealer.value < player.value <= 21 and dealer.value < secondaryplayer.value <= 21:
        print('Both hands win')
        chips.total = chips.total + chips.bet*2
        
    elif dealer.value < player.value <= 21 and secondaryplayer.value > 21:
        print('Main hand wins. Split hand busted')
        
    
    elif dealer.value < secondaryplayer.value <= 21 and player.value > 21:
        print('Split hand wins. Main hand busted')
    
    
    elif player.value > 21 and secondaryplayer.value > 21:
        print('Both hands busted')
        chips.total = chips.total - chips.bet*2
    
    
    elif player.value == secondaryplayer.value == dealer.value <= 21:
        print('Both hands push with dealer')
        
    
    elif dealer.value < player.value <= 21 and dealer.value == secondaryplayer.value:
        print('Main hands wins. Split hand pushes with dealer')
        chips.total = chips.total + chips.bet
        
    
    elif dealer.value < secondaryplayer.value <= 21 and dealer.value == player.value:
        print('Split hand wins. Main hand pushes with dealer')
        chips.total = chips.total + chips.bet
    
    
    elif secondaryplayer.value < player.value == dealer.value <= 21:
        print('Main hand pushes with dealer. Secondary hand loses')
        chips.total = chips.total - chips.bet
        
    
    elif player.value < secondaryplayer.value == dealer.value <= 21:
        print('Secondary hand pushes with dealer. Main hand loses')
        chips.total = chips.total - chips.bet
        
    
    elif player.value == dealer.value <= 21 and secondaryplayer.value > 21:
        print('Main hand pushes with dealer. Split hand busted')
        chips.total = chips.total - chips.bet
        
    
    elif secondaryplayer.value == dealer.value <= 21 and player.value > 21:
        print('Split hand pushes with dealer. Main hand busted')
        chips.total = chips.total - chips.bet
        
    
    elif player.value < dealer.value <= 21 and  secondaryplayer.value < dealer.value <= 21:
        print('Both hands lose. Dealer wins')
        chips.total = chips.total - chips.bet*2
        
    
    elif secondaryplayer.value < dealer.value < player.value <= 21:
        print('Main hand wins. Split hand loses')
        
    
    elif player.value < dealer.value < secondaryplayer.value <= 21:
        print('Split hand wins. Main hand loses')
    
    
    elif player.value <= 21 and secondaryplayer.value > 21 and dealer.value > 21:
        print('Main hand wins. Split hand and dealer busted')
        
    
    elif secondaryplayer.value <= 21 and player.value > 21 and dealer.value > 21:
        print('Split hand wins. Main hand and dealer busted')
        
    
    elif player.value <= 21 and secondaryplayer.value <= 21 and dealer.value > 21:
        print('Both hands win. Dealer busted')
        chips.total = chips.total + chips.bet*2
    
    
    elif secondaryplayer.value > 21 and player.value < dealer.value <= 21:
        print('Main hand loses. Split hand busted')
        chips.total = chips.total - chips.bet*2
    
    
    elif player.value > 21 and secondaryplayer.value < dealer.value <= 21:
        print('Main hand busted. Split hand loses')
        chips.total = chips.total - chips.bet*2
    
    
    else:
        print('End not recognized. Sorry for the incovenience. Nothing will happen')
        
    
    
    print(f'\nYou have {chips.total} chips')
    ask_for_playagain()
        
        
    

In [268]:
def on_button_clicked_a(b):
    # button a for starting the game
    buttona.close()
    buttonb.close()
    initialize_game()

In [269]:
def on_button_clicked_b(b):
    # button b to quit playing
    print('Thanks for playing!')
    buttona.close()
    buttonb.close()

In [270]:
def on_button_clicked_c(b):
    # button c to place the bet showing in the slider at the moment of clicking
    slidera.close()
    buttonc.close()
    take_bet()

In [271]:
def on_button_clicked_d(b):
    global shoe, player
    # button d to hit
    hit(shoe,player)
    buttond.close()
    buttone.close()
    display.clear_output()
    player_rounds()
    

In [272]:
def on_button_clicked_e(b):
    # button e to stand
    buttond.close()
    buttone.close()
    display.clear_output()
    dealer_rounds()
    
    

In [273]:
def on_button_clicked_f(b):
    # button f to play again next round
    buttonf.close()
    buttong.close()
    display.clear_output()
    initialize_game()
    

In [274]:
def on_button_clicked_g(b):
    # button g to quit playing after being played at least one round
    buttonf.close()
    buttong.close()
    display.clear_output()
    print(f'Thanks for playing. You leave with {chips.total} chips.')

In [275]:
def on_button_clicked_h(b):
    # button h to double down
    buttond.close()
    buttone.close()
    buttonh.close()
    display.clear_output()
    double_down_hit()

In [276]:
def on_button_clicked_i(b):
    # button i to hit in case of double down
    buttoni.close()
    display.clear_output()
    hit(shoe, player)
    double_down_stand(shoe,player)

In [277]:
def on_button_clicked_j(b):
    global secondaryplayer, secondarybet
    #button j creates splithand and draws two cards adjusting for aces, crates splitbet and starts the splitgame
    buttonj.close()
    display.clear_output()
    secondaryplayer = Hand()
    secondarybet = chips.bet
    secondaryplayer.add_card(shoe.deal())
    secondaryplayer.add_card(shoe.deal())
    secondaryplayer.adjust_for_ace()
    player_rounds_main()

In [278]:
def on_button_clicked_k(b):
    global shoe, player
    # button d to hit main hand
    hit(shoe,player)
    display.clear_output()
    player_rounds_main()

In [279]:
def on_button_clicked_l(b):
    # button l to stand in main hand
    display.clear_output()
    player_rounds_secondary()

In [281]:
def on_button_clicked_n(b):
    global shoe, secondaryplayer
    # button n to hit for split hand
    display.clear_output()
    hit(shoe, secondaryplayer)
    player_rounds_secondary()

In [282]:
def on_button_clicked_o(b):
    # button o to stand for split hand, goes to dealer_split
    display.clear_output()
    dealer_rounds_split()

In [343]:
def on_button_clicked_m(b):
    display.clear_output()
    print('Player surrenders. Half of the bet returned')
    chips.total = int(chips.total - chips.bet/2)
    print(f'\nYou have now {chips.total} chips ')
    ask_for_playagain()

In [348]:
def on_button_clicked_p(b):
    display.clear_output()
    if dealer.value == 21:
        chips.total = chips.total + chips.bet*2
        print(f'Dealer has a blackjack! Insurance pays double the bet.\nYou have now {chips.total} chips')
    else:
        chips.total = chips.total - chips.bet
        print(f"Dealer doesn't have a blackjack! You lose the bet.\n You have now {chips.total} chips")
    ask_for_playagain()

In [284]:
def start_playing():
    global buttona, buttonb, audio
    
    # shows the first main buttons and some info
    
    print('BACKJACK 0.3  (Now with real casino rules!)')
    #audio
    display.display(buttona)
    display.display(buttonb) 
    buttona.on_click(on_button_clicked_a)
    buttonb.on_click(on_button_clicked_b)
    print('')
    print('')
    print('by jaume')
    print('2020')
    #play_audio()

In [285]:
def play():
    global buttona, buttonb, chips, ft, audio

    # to play for the first time. Initializes the chips, variable first time and  then starts
    buttona = widgets.Button(description = 'Start playing!', button_style = 'info')
    buttonb = widgets.Button(description = 'Quit', button_style = 'info')
    chips = Chips()
    ft = True
    start_playing()
    

In [286]:
def play_audio():
    
    # creates an audio object to play during the game
    global audio
    audio = display.Audio('8ball.wav', autoplay=True)
    print('playing audio')
    return audio

In [377]:
def chance_busting():
    pass

In [376]:
def suggested_move():
    pass

In [375]:
def count():
    pass

In [287]:
#play_audio()

In [374]:
play()

Thanks for playing. You leave with 166 chips.
